In [1]:
from VariationalAutoDecoder_Gumbel import VariationalAutoDecoder as VAD_Gumbel
from VAD_Trainer import VAD_Trainer
import utils
from evaluate import evaluate_model
import torch
import torch.optim as optim
import torch.nn as nn
import csv
import time
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Create DataLoaders

In [2]:
train_ds, train_dl, test_ds, test_dl = utils.create_dataloaders(data_path="dataset" ,batch_size=64)

## Train Auto Decoder

In [ ]:
import torch.nn as nn

architectures = [
    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    },

    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    },

    {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    }
]



# latent_dims = [dim for dim in [64, 32, 128, 16, 10] for _ in range(5)]
VADs = [VAD(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device) for arch in architectures]# for _ in range(5)]
# learning_rates = [lr for lr in [0.001, 0.0005, 0.0001, 0.002, 0.005] for _ in range(5)]
trainers = [VAD_Trainer(var_decoder=VADs[i], dataloader=train_dl, latent_dim=128, device=device, lr=1e-2) for i in range(len(VADs))]

In [ ]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
temp_latents = torch.randn(10, 128).to(device)
latents_list = [torch.nn.Parameter(torch.stack([temp_latents[label,:] for label in train_dl.dataset.y])).to(device) for i in range(len(VADs))]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list]

# Save results to a CSV file
csv_file_path = 'results_VAD_temp.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(500)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=500)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=VADs[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list[index], epochs=500, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

In [ ]:
for i in range(len(trainers)):
    utils.plot_tsne(train_ds, trainers[i].latents, f"tsne_kl_{i}")

## Fine Tune

In [ ]:
arch = {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    }

learning_rates = [0.001, 0.0001, 0.005]
VADs_ft = [VAD(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device) for _ in range(len(learning_rates))]# for _ in range(5)]
trainers_ft = [VAD_Trainer(var_decoder=VADs_ft[i], dataloader=train_dl, latent_dim=128, device=device, lr=learning_rates[i]) for i in range(len(VADs_ft))]

In [ ]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
temp_latents_ft = torch.randn(10, 128).to(device)
latents_list_ft = [torch.nn.Parameter(torch.stack([temp_latents_ft[label,:] for label in train_dl.dataset.y])).to(device) for i in range(len(VADs_ft))]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list_ft]

# Save results to a CSV file
csv_file_path = 'results_VAD_ft.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(500)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers_ft):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=500)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=VADs_ft[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list_ft[index], epochs=500, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

In [ ]:
for i in range(len(trainers_ft)):
    utils.plot_tsne(train_ds, trainers_ft[i].latents, f"tsne_kl_ft_{i}")

## Best model

In [3]:
arch = {
        "mu_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        ),
        "log_var_net": nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128)
        )
    }

VAD_best = VAD_Gumbel(mu_layers=arch['mu_net'], var_layers=arch['log_var_net'], device=device)
trainer_best = VAD_Trainer(var_decoder=VAD_best, dataloader=train_dl, latent_dim=128, device=device, lr=0.001)
trainer_best.train(num_epochs=1000)

Epoch [1/1000], Loss: 0.4888
Epoch [2/1000], Loss: 0.4584
Epoch [3/1000], Loss: 0.4350
Epoch [4/1000], Loss: 0.4123
Epoch [5/1000], Loss: 0.3900
Epoch [6/1000], Loss: 0.3673
Epoch [7/1000], Loss: 0.3454
Epoch [8/1000], Loss: 0.3254
Epoch [9/1000], Loss: 0.3069
Epoch [10/1000], Loss: 0.2935
Epoch [11/1000], Loss: 0.2827
Epoch [12/1000], Loss: 0.2735
Epoch [13/1000], Loss: 0.2671
Epoch [14/1000], Loss: 0.2610
Epoch [15/1000], Loss: 0.2574
Epoch [16/1000], Loss: 0.2544
Epoch [17/1000], Loss: 0.2546
Epoch [18/1000], Loss: 0.2495
Epoch [19/1000], Loss: 0.2471
Epoch [20/1000], Loss: 0.2441
Epoch [21/1000], Loss: 0.2421
Epoch [22/1000], Loss: 0.2394
Epoch [23/1000], Loss: 0.2379
Epoch [24/1000], Loss: 0.2366
Epoch [25/1000], Loss: 0.2376
Epoch [26/1000], Loss: 0.2384
Epoch [27/1000], Loss: 0.2360
Epoch [28/1000], Loss: 0.2325
Epoch [29/1000], Loss: 0.2272
Epoch [30/1000], Loss: 0.2242
Epoch [31/1000], Loss: 0.2221
Epoch [32/1000], Loss: 0.2206
Epoch [33/1000], Loss: 0.2191
Epoch [34/1000], Lo

[0.48879022896289825,
 0.4583757072687149,
 0.4350278601050377,
 0.41225976683199406,
 0.3900135736912489,
 0.3672998361289501,
 0.3454465363174677,
 0.32537527941167355,
 0.30685845017433167,
 0.2935327049344778,
 0.28268183395266533,
 0.2734734546393156,
 0.2671023244038224,
 0.2610362060368061,
 0.25739262718707323,
 0.25440324656665325,
 0.2545580752193928,
 0.24947510194033384,
 0.24712234176695347,
 0.2441177824512124,
 0.24208341352641582,
 0.23943259567022324,
 0.23791407886892557,
 0.23658426851034164,
 0.237641136161983,
 0.23843551147729158,
 0.23596907127648592,
 0.23245725221931934,
 0.2272246303036809,
 0.22415499109774828,
 0.22206301148980856,
 0.22058347705751657,
 0.21909974422305822,
 0.21759890858083963,
 0.21681061573326588,
 0.21733608096837997,
 0.21654117479920387,
 0.21704340539872646,
 0.21742731239646673,
 0.21698286291211843,
 0.21528511587530375,
 0.21434705704450607,
 0.21353930793702602,
 0.21385412570089102,
 0.21327823493629694,
 0.2115912763401866,
 0.

In [4]:
utils.plot_tsne(train_ds, trainer_best.latents, f"tsne_Gumbel")

<Figure size 800x600 with 0 Axes>

## Sample specific vectors

In [4]:
num_test_samples = len(test_dl.dataset)
temp_latents_best = torch.randn(10, 128).to(device)
latents_best = torch.nn.Parameter(torch.stack([temp_latents_best[label,:] for label in test_dl.dataset.y])).to(device)
opt = optim.Adam([latents_best], lr=1e-3)

In [5]:
test_loss = evaluate_model(model=VAD_best, test_dl=test_dl, opt=opt, latents=latents_best, epochs=1000, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

AD has finished test evaluation with a test loss of 0.15479616168886423.


In [7]:
utils.plot_tsne(test_ds, latents_best, f"tsne_test_Gumbel")

<Figure size 800x600 with 0 Axes>

In [8]:
# Randomly sample 5 indices from the test dataset
random.seed(6)
sampled_indices = random.sample(range(len(latents_best)), 5)

# Extract the corresponding vectors (input data) and their labels
sampled_latents = [latents_best[i] for i in sampled_indices]  # Only selecting input data, not labels

# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.randn_like(sampled_latents_tensor)

sampled_test_images = VAD_best(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = VAD_best(random_latents_tensor).view(-1, 1, 28, 28)

utils.save_images(sampled_test_images, "sampled_test_images_VAD_Gumbel.png")
utils.save_images(random_test_images, "random_test_images_VAD_Gumbel.png")

## Interpolation

In [6]:
import numpy as np
random.seed(44)
sampled_indices = random.sample(range(len(latents_best)), 2)
sampled_latents = [latents_best[i] for i in sampled_indices]
weights = np.linspace(0, 1, 7)
interpolated_latents = [w * sampled_latents[0] + (1 - w) * sampled_latents[1] for w in weights]
interpolated_latents_tensor = torch.stack(interpolated_latents)
interpolated_images = VAD_best(interpolated_latents_tensor).view(-1, 1, 28, 28)
utils.save_images(interpolated_images, "interpolated_images_gumbell.png")